In this notebook we will be testing zero-shot / few-shot prompting

In [1]:
import sys
sys.path.append("..")
from llama_cpp import Llama
import pandas as pd
from langdetect import detect
from mlflow_experiment import MlflowExperiment, EvaluationPipeline, InferencePipeline
from mlflow_experiment.evaluation.metrics import (accuracy_score, 
                                                  precision,
                                                  recall,
                                                  false_negative_rate,
                                                  false_positive_rate,
                                                  median_response_token_count,
                                                  median_query_token_count, 
                                                  median_processing_time, 
                                                  median_tokens_per_second, 
                                                  hallucination_rate, 
                                                  bad_output_format_rate)
from mlflow_experiment.inference.prompt_building.basic_user_prompt_builder import (
    BasicUserPromptBuilder,
)
from mlflow_experiment.inference.postprocessing.basic_postprocessing import (
    BasicPostprocessing,
)
from mlflow_experiment.inference.model.basic_llama_cpp_model import BasicLlamaCppModel
import dotenv
import pandas as pd
from itertools import product

In [2]:
dotenv.load_dotenv("../.env",override=True)

True

In [15]:
data = pd.read_json("../data/subsample.jsonl", lines = True)

In [16]:
ev_pipeline = EvaluationPipeline(accuracy_score,
                                 precision,
                                 recall,
                                 false_negative_rate,
                                 false_positive_rate,
                                 median_response_token_count,
                                 median_query_token_count,
                                 median_processing_time,
                                 median_tokens_per_second,
                                 hallucination_rate,
                                 bad_output_format_rate,
                                 
                                )

In [17]:
MODEL_NAME = "Qwen2.5-1.5B-Instruct-Q5_K_M.gguf"
MODEL_PATH = f"../models/{MODEL_NAME}"
model = Llama(MODEL_PATH, verbose=False, n_ctx=32768, n_gpu_layers=-1, n_batch=2048)

prompt_builder = BasicUserPromptBuilder(
    """This is a movie review: {review}.
    ----
    Determine whether the overall sentiment of the review is positive or negative. You must look for evaluatory statements about the movie itself - e.g. "thrilling", "terrible acting", etc. These evaluatory statements are distinct from the movie genre! So be careful not to mistake descriptions about the genre. Take into account that different evaluatory statements might have different connation for different genres - e.g. a "terrifying movie" in the horror genre means a perfect movie, but in some other genre like commedy it might very well mean - rubbish movie.
    ----
    Your output: your output should be either "positive" (if the overall sentiment is positive) or "negative" (if the overall sentiment is negative). Do not output nothing more. Output only "positive" or "negative"!
    """
)
llama_model = BasicLlamaCppModel(model, user_prompt_builder=prompt_builder, model_name = MODEL_NAME)

postprocessing_fn = BasicPostprocessing({"negative": 1, "positive": 0})

inf_pipeline = InferencePipeline(
    llama_model,
    system_prompt="You are a sentiment analysis system. Your goal is to categorize movie reviews into positive or negative. Follow the instructions given precisely!",
    postprocessing_fn=postprocessing_fn,
)



In [18]:
exp = MlflowExperiment(
    inference_pipeline=inf_pipeline,
    evaluation_pipeline=ev_pipeline,
    experiment_name="zero-shot-few-shot-prompting",
)

In [19]:
temperature, top_k, top_p = 0.2, 40, 0.95
#for temperature, top_k, top_p in product([0.2, 0.5, 1.0], [40, 500, 0], [0.95, 0.5, 0.0]):
inference_outputs, evaluation_results = exp.run(
    data[["review"]].to_dict(orient="records"),
    y_true=data.label,
    experiment_run_tags={"justification": "Provide description of the task without concrete examples. "},
    run_name=f"zero-shot-basic",
    temperature = temperature,
    top_k = top_k,
    top_p = top_p
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [09:04<00:00, 18.37it/s]

🏃 View run FULL_DATASET_zero-shot-basic at: http://localhost:5000/#/experiments/173405608153165389/runs/0c8d2551607b4c65abe3079da3c8770d
🧪 View experiment at: http://localhost:5000/#/experiments/173405608153165389


In [20]:
evaluation_results

{'accuracy_score': 0.9429,
 'precision': np.float64(0.9531409862901575),
 'recall': np.float64(0.9316),
 'false_negative_rate': np.float64(0.06840000000000002),
 'false_positive_rate': np.float64(0.0458),
 'median_response_token_count': np.float64(1.0),
 'median_query_token_count': np.float64(333.5),
 'median_processing_time': np.float64(0.04989945888519287),
 'median_tokens_per_second': np.float64(40.08059415280785),
 'hallucination_rate': np.float64(0.0),
 'bad_output_format_rate': np.float64(0.0)}

In [19]:
inference_outputs.loc[:, "true"] = data.label

In [20]:
inference_outputs

,output_string,label,is_hallucination,is_wrong_format,input_token_count,output_token_count,elapsed_time,true
0,positive,0,False,False,290,1,0.046078,0
1,positive,0,False,False,469,1,0.040344,1
2,negative,1,False,False,508,1,0.049691,1
3,positive,0,False,False,285,1,0.034716,0
4,positive,0,False,False,439,1,0.039175,0
...,...,...,...,...,...,...,...,...
1116,positive,0,False,False,263,1,0.026869,1
1117,positive,0,False,False,442,1,0.032223,0
1118,positive,0,False,False,669,1,0.045406,1
1119,positive,0,False,False,443,1,0.034034,0
